# Generación de Data Única referente a Cortes , tribunales y Regiones

Transformación de algunos datos para normalizar nombres en columnas, además de unificacion por comunas con data de Censo

In [53]:
import pandas as pd
import numpy as np
import os

from tqdm import tqdm
from unicodedata import normalize

### Analizo Data de Tribunales y Cortes de Apelaciones

In [54]:
path_processed = "../data/processed/pjud"
path_censo = "../data/processed/censo"

In [55]:
df_regiones = pd.read_feather(f"{path_processed}/processes_TerminosRol.feather")
df_tribunales = pd.read_feather(f"{path_processed}/generates_ListadoTribunales.feather")
df_censo = pd.read_feather(f"{path_censo}/generates_Censo2017.feather")


In [56]:
df_tribunales

,index,REGION,TRIBUNAL,ASIENTO,COMUNA,JUECES,TIPO JUZGADO
0,0,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,HUARA,13,ORAL
1,1,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,CAMIÑA,13,ORAL
2,2,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,COLCHANE,13,ORAL
3,3,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,IQUIQUE,13,ORAL
4,4,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,POZO ALMONTE,13,ORAL
...,...,...,...,...,...,...,...
687,341,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,PRIMAVERA,1,LETRAS Y GARANTIA
688,342,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,TIMAUKEL,1,LETRAS Y GARANTIA
689,343,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,CABO DE HORNOS,1,LETRAS Y GARANTIA
690,344,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,ANTARTICA,1,LETRAS Y GARANTIA


In [57]:
# Extraigo Cortes de Apelaciones asociadas a Juzgados desde este DataSet
data_cortes_apelaciones = pd.unique(df_regiones[['CORTE','TRIBUNAL']].values.ravel())
cortes_tribunales = []

for data in range(len(data_cortes_apelaciones)):  
    if not data_cortes_apelaciones[data].find('C.A.') == -1:
        corte_apelacion = data_cortes_apelaciones[data]
    else:
        tribunal = data_cortes_apelaciones[data]
        if tribunal.find("TRIBUNAL") != -1:
            separa_ciudad = tribunal.split("PENAL ")
        else:
            separa_ciudad = tribunal.split("GARANTIA ")
        ciudad = separa_ciudad[1]
        cortes_tribunales.append([corte_apelacion,ciudad,tribunal])

#cortes_tribunales

In [58]:
lista_cortes = []

for trib in df_tribunales.index:
    for indice in range(len(cortes_tribunales)):
        if df_tribunales['TRIBUNAL'][trib] == cortes_tribunales[indice][2]:
            corte = cortes_tribunales[indice][0]
            lista_cortes.append(corte)
            break

df_tribunales_cortes = df_tribunales.copy()
df_tribunales_cortes['CORTE'] = lista_cortes

In [59]:
df_tribunales

,index,REGION,TRIBUNAL,ASIENTO,COMUNA,JUECES,TIPO JUZGADO
0,0,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,HUARA,13,ORAL
1,1,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,CAMIÑA,13,ORAL
2,2,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,COLCHANE,13,ORAL
3,3,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,IQUIQUE,13,ORAL
4,4,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,POZO ALMONTE,13,ORAL
...,...,...,...,...,...,...,...
687,341,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,PRIMAVERA,1,LETRAS Y GARANTIA
688,342,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,TIMAUKEL,1,LETRAS Y GARANTIA
689,343,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,CABO DE HORNOS,1,LETRAS Y GARANTIA
690,344,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,ANTARTICA,1,LETRAS Y GARANTIA


In [60]:
df_tribunales_cortes

,index,REGION,TRIBUNAL,ASIENTO,COMUNA,JUECES,TIPO JUZGADO,CORTE
0,0,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,HUARA,13,ORAL,C.A. DE IQUIQUE
1,1,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,CAMIÑA,13,ORAL,C.A. DE IQUIQUE
2,2,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,COLCHANE,13,ORAL,C.A. DE IQUIQUE
3,3,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,IQUIQUE,13,ORAL,C.A. DE IQUIQUE
4,4,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,POZO ALMONTE,13,ORAL,C.A. DE IQUIQUE
...,...,...,...,...,...,...,...,...
687,341,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,PRIMAVERA,1,LETRAS Y GARANTIA,C.A. DE PUNTA ARENAS
688,342,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,TIMAUKEL,1,LETRAS Y GARANTIA,C.A. DE PUNTA ARENAS
689,343,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,CABO DE HORNOS,1,LETRAS Y GARANTIA,C.A. DE PUNTA ARENAS
690,344,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,ANTARTICA,1,LETRAS Y GARANTIA,C.A. DE PUNTA ARENAS


In [61]:
df_tribunales_cortes.query("CORTE == 'C.A. DE SAN MIGUEL'")

,index,REGION,TRIBUNAL,ASIENTO,COMUNA,JUECES,TIPO JUZGADO,CORTE
265,265,REGION METROPOLITANA DE SANTIAGO,TRIBUNAL DE JUICIO ORAL EN LO PENAL PUENTE ALTO,PUENTE ALTO,PUENTE ALTO,9,ORAL,C.A. DE SAN MIGUEL
266,266,REGION METROPOLITANA DE SANTIAGO,TRIBUNAL DE JUICIO ORAL EN LO PENAL PUENTE ALTO,PUENTE ALTO,SAN JOSE DE MAIPO,9,ORAL,C.A. DE SAN MIGUEL
267,267,REGION METROPOLITANA DE SANTIAGO,TRIBUNAL DE JUICIO ORAL EN LO PENAL PUENTE ALTO,PUENTE ALTO,PIRQUE,9,ORAL,C.A. DE SAN MIGUEL
268,268,REGION METROPOLITANA DE SANTIAGO,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,SAN BERNARDO,SAN BERNARDO,9,ORAL,C.A. DE SAN MIGUEL
269,269,REGION METROPOLITANA DE SANTIAGO,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,SAN BERNARDO,CALERA DE TANGO,9,ORAL,C.A. DE SAN MIGUEL
270,270,REGION METROPOLITANA DE SANTIAGO,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,SAN BERNARDO,BUIN,9,ORAL,C.A. DE SAN MIGUEL
271,271,REGION METROPOLITANA DE SANTIAGO,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,SAN BERNARDO,PAINE,9,ORAL,C.A. DE SAN MIGUEL
272,272,REGION METROPOLITANA DE SANTIAGO,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,MELIPILLA,MELIPILLA,6,ORAL,C.A. DE SAN MIGUEL
273,273,REGION METROPOLITANA DE SANTIAGO,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,MELIPILLA,SAN PEDRO,6,ORAL,C.A. DE SAN MIGUEL
274,274,REGION METROPOLITANA DE SANTIAGO,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,MELIPILLA,ALHUE,6,ORAL,C.A. DE SAN MIGUEL


In [62]:
df_censo.columns

Index(['index', 'ORDEN', 'NOMBRE REGIÓN', 'CÓDIGO REGIÓN', 'NOMBRE PROVINCIA',
       'CÓDIGO PROVINCIA', 'NOMBRE COMUNA', 'CÓDIGO COMUNA', 'EDAD',
       'TOTAL POBLACIÓN EFECTIVAMENTE CENSADA', 'HOMBRES ', 'MUJERES',
       'TOTAL ÁREA URBANA', 'HOMBRES ÁREA URBANA ', 'MUJERES ÁREA URBANA',
       'TOTAL ÁREA RURAL', 'HOMBRES ÁREA RURAL', 'MUJERES ÁREA RURAL'],
      dtype='object')

In [63]:
poblacion = []

for trib in df_tribunales.index:
    for indice in df_censo.index:
        if df_tribunales['COMUNA'][trib] == df_censo['NOMBRE COMUNA'][indice]:
            censado = df_censo['TOTAL POBLACIÓN EFECTIVAMENTE CENSADA'][indice]
            hombres = df_censo['HOMBRES '][indice]
            mujeres = df_censo['MUJERES'][indice]
            urbana = df_censo['TOTAL ÁREA URBANA'][indice]
            rural = df_censo['TOTAL ÁREA RURAL'][indice]
            poblacion.append([censado, hombres, mujeres, urbana, rural])
            break

df_poblacion = pd.DataFrame(poblacion, columns = ['POBLACION', 'HOMBRES', 'MUJERES', 'URBANO', 'RURAL'])

In [64]:
df_poblacion

,POBLACION,HOMBRES,MUJERES,URBANO,RURAL
0,2730,1501,1229,1109,1621
1,1250,657,593,0,1250
2,1728,995,733,0,1728
3,191468,94897,96571,189065,2403
4,15711,8987,6724,10095,5616
...,...,...,...,...,...
687,1158,930,228,0,1158
688,405,343,62,0,405
689,2063,1195,868,1874,189
690,138,126,12,0,138


In [65]:
df_tribunales_poblacion = pd.concat([df_tribunales_cortes, df_poblacion.reindex(df_tribunales_cortes.index)], axis=1)

In [66]:
df_tribunales_poblacion

,index,REGION,TRIBUNAL,ASIENTO,COMUNA,JUECES,TIPO JUZGADO,CORTE,POBLACION,HOMBRES,MUJERES,URBANO,RURAL
0,0,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,HUARA,13,ORAL,C.A. DE IQUIQUE,2730,1501,1229,1109,1621
1,1,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,CAMIÑA,13,ORAL,C.A. DE IQUIQUE,1250,657,593,0,1250
2,2,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,COLCHANE,13,ORAL,C.A. DE IQUIQUE,1728,995,733,0,1728
3,3,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,IQUIQUE,13,ORAL,C.A. DE IQUIQUE,191468,94897,96571,189065,2403
4,4,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,POZO ALMONTE,13,ORAL,C.A. DE IQUIQUE,15711,8987,6724,10095,5616
...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,341,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,PRIMAVERA,1,LETRAS Y GARANTIA,C.A. DE PUNTA ARENAS,1158,930,228,0,1158
688,342,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,TIMAUKEL,1,LETRAS Y GARANTIA,C.A. DE PUNTA ARENAS,405,343,62,0,405
689,343,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,CABO DE HORNOS,1,LETRAS Y GARANTIA,C.A. DE PUNTA ARENAS,2063,1195,868,1874,189
690,344,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,ANTARTICA,1,LETRAS Y GARANTIA,C.A. DE PUNTA ARENAS,138,126,12,0,138


Creo un dataset con informacion de poblacion que abarca cada tribunal !!!

In [67]:
tribunales = df_tribunales['TRIBUNAL'].unique()
poblacion_tribunal = []

for trib in range(len(tribunales)):  
    poblacion = 0
    hombres = 0
    mujeres = 0
    urbana = 0
    rural = 0
    comunas = []
    for indice in df_tribunales_poblacion.index:
        if tribunales[trib] == df_tribunales_poblacion['TRIBUNAL'][indice]:
            region = df_tribunales_poblacion['REGION'][indice]
            corte = df_tribunales_poblacion['CORTE'][indice]
            tribunal = df_tribunales_poblacion['TRIBUNAL'][indice]
            poblacion = int(df_tribunales_poblacion['POBLACION'][indice]) + poblacion
            hombres = int(df_tribunales_poblacion['HOMBRES'][indice]) + hombres
            mujeres = int(df_tribunales_poblacion['MUJERES'][indice]) + mujeres
            urbana = int(df_tribunales_poblacion['URBANO'][indice]) + urbana
            rural = int(df_tribunales_poblacion['RURAL'][indice]) + rural
            comunas.append(df_tribunales_poblacion['COMUNA'][indice])
            
    poblacion_tribunal.append([region, corte, tribunal, poblacion, hombres, mujeres, urbana, rural, comunas])
    
    
columnas = ['REGION', 'CORTE', 'TRIBUNAL', 'POBLACION', 'HOMBRES', 'MUJERES', 'URBANO', 'RURAL', 'COMUNAS']
df_poblacion_jurisdiccion = pd.DataFrame(poblacion_tribunal, columns = columnas)

In [68]:
df_poblacion_jurisdiccion

,REGION,CORTE,TRIBUNAL,POBLACION,HOMBRES,MUJERES,URBANO,RURAL,COMUNAS
0,REGION DE TARAPACA,C.A. DE IQUIQUE,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,330558,167793,162765,310065,20493,"[HUARA, CAMIÑA, COLCHANE, IQUIQUE, POZO ALMONT..."
1,REGION DE ANTOFAGASTA,C.A. DE ANTOFAGASTA,TRIBUNAL DE JUICIO ORAL EN LO PENAL CALAMA,177048,92417,84631,164011,13037,"[CALAMA, OLLAGUE, SAN PEDRO DE ATACAMA]"
2,REGION DE ANTOFAGASTA,C.A. DE ANTOFAGASTA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ANTOFAGASTA,430486,222597,207889,407737,22749,"[TOCOPILLA, MARIA ELENA, MEJILLONES, SIERRA GO..."
3,REGION DE ATACAMA,C.A. DE COPIAPO,TRIBUNAL DE JUICIO ORAL EN LO PENAL COPIAPO,286168,144420,141748,260520,25648,"[CHAÑARAL, DIEGO DE ALMAGRO, CALDERA, COPIAPO,..."
4,REGION DE COQUIMBO,C.A. DE LA SERENA,TRIBUNAL DE JUICIO ORAL EN LO PENAL LA SERENA,496337,239495,256842,443484,52853,"[LA HIGUERA, VICUÑA, LA SERENA, COQUIMBO, ANDA..."
...,...,...,...,...,...,...,...,...,...
192,REGION DE AISEN DEL GENERAL CARLOS IBAÑEZ DEL ...,C.A. DE COYHAIQUE,JUZGADO DE LETRAS Y GARANTIA COCHRANE,4638,2561,2077,2841,1797,"[COCHRANE, O'HIGGINS, TORTEL]"
193,REGION DE MAGALLANES Y ANTARTICA CHILENA,C.A. DE PUNTA ARENAS,JUZGADO DE LETRAS Y GARANTIA NATALES,22686,11792,10894,19180,3506,"[NATALES, TORRES DEL PAINE]"
194,REGION DE MAGALLANES Y ANTARTICA CHILENA,C.A. DE PUNTA ARENAS,JUZGADO DE LETRAS Y GARANTIA PORVENIR,8364,5266,3098,6062,2302,"[PORVENIR, PRIMAVERA, TIMAUKEL]"
195,REGION DE MAGALLANES Y ANTARTICA CHILENA,C.A. DE PUNTA ARENAS,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,2201,1321,880,1874,327,"[CABO DE HORNOS, ANTARTICA]"


Agregare data de cantidad de jueces a esta Data y Asientio de cada Tribunal.

In [69]:
dotacion = []
i=0
for indice in df_poblacion_jurisdiccion.index:
    for trib in df_tribunales.index:
        if df_poblacion_jurisdiccion['TRIBUNAL'][indice] == df_tribunales['TRIBUNAL'][trib]:
            jueces = df_tribunales['JUECES'][trib]
            asiento = df_tribunales['ASIENTO'][trib]
            tipo = df_tribunales['TIPO JUZGADO'][trib]
            row = [jueces, asiento, tipo]
            dotacion.append(row)
            break
columnas = ['JUECES','ASIENTO','TIPO JUZGADO']
df_anexo = pd.DataFrame(dotacion, columns=columnas)

In [70]:
df_poblacion_jurisdiccion = pd.concat([df_poblacion_jurisdiccion,df_anexo], axis=1)

In [71]:
df_poblacion_jurisdiccion

,REGION,CORTE,TRIBUNAL,POBLACION,HOMBRES,MUJERES,URBANO,RURAL,COMUNAS,JUECES,ASIENTO,TIPO JUZGADO
0,REGION DE TARAPACA,C.A. DE IQUIQUE,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,330558,167793,162765,310065,20493,"[HUARA, CAMIÑA, COLCHANE, IQUIQUE, POZO ALMONT...",13,IQUIQUE,ORAL
1,REGION DE ANTOFAGASTA,C.A. DE ANTOFAGASTA,TRIBUNAL DE JUICIO ORAL EN LO PENAL CALAMA,177048,92417,84631,164011,13037,"[CALAMA, OLLAGUE, SAN PEDRO DE ATACAMA]",7,CALAMA,ORAL
2,REGION DE ANTOFAGASTA,C.A. DE ANTOFAGASTA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ANTOFAGASTA,430486,222597,207889,407737,22749,"[TOCOPILLA, MARIA ELENA, MEJILLONES, SIERRA GO...",13,ANTOFAGASTA,ORAL
3,REGION DE ATACAMA,C.A. DE COPIAPO,TRIBUNAL DE JUICIO ORAL EN LO PENAL COPIAPO,286168,144420,141748,260520,25648,"[CHAÑARAL, DIEGO DE ALMAGRO, CALDERA, COPIAPO,...",9,COPIAPO,ORAL
4,REGION DE COQUIMBO,C.A. DE LA SERENA,TRIBUNAL DE JUICIO ORAL EN LO PENAL LA SERENA,496337,239495,256842,443484,52853,"[LA HIGUERA, VICUÑA, LA SERENA, COQUIMBO, ANDA...",10,LA SERENA,ORAL
...,...,...,...,...,...,...,...,...,...,...,...,...
192,REGION DE AISEN DEL GENERAL CARLOS IBAÑEZ DEL ...,C.A. DE COYHAIQUE,JUZGADO DE LETRAS Y GARANTIA COCHRANE,4638,2561,2077,2841,1797,"[COCHRANE, O'HIGGINS, TORTEL]",1,COCHRANE,LETRAS Y GARANTIA
193,REGION DE MAGALLANES Y ANTARTICA CHILENA,C.A. DE PUNTA ARENAS,JUZGADO DE LETRAS Y GARANTIA NATALES,22686,11792,10894,19180,3506,"[NATALES, TORRES DEL PAINE]",2,NATALES,LETRAS Y GARANTIA
194,REGION DE MAGALLANES Y ANTARTICA CHILENA,C.A. DE PUNTA ARENAS,JUZGADO DE LETRAS Y GARANTIA PORVENIR,8364,5266,3098,6062,2302,"[PORVENIR, PRIMAVERA, TIMAUKEL]",1,PORVENIR,LETRAS Y GARANTIA
195,REGION DE MAGALLANES Y ANTARTICA CHILENA,C.A. DE PUNTA ARENAS,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,2201,1321,880,1874,327,"[CABO DE HORNOS, ANTARTICA]",1,CABO DE HORNOS,LETRAS Y GARANTIA


In [72]:
df_poblacion_jurisdiccion.query("CORTE == 'C.A. DE SAN MIGUEL'")

,REGION,CORTE,TRIBUNAL,POBLACION,HOMBRES,MUJERES,URBANO,RURAL,COMUNAS,JUECES,ASIENTO,TIPO JUZGADO
31,REGION METROPOLITANA DE SANTIAGO,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL PUENTE ALTO,612816,298437,314379,590972,21844,"[PUENTE ALTO, SAN JOSE DE MAIPO, PIRQUE]",9,PUENTE ALTO,ORAL
32,REGION METROPOLITANA DE SANTIAGO,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,496078,244614,251464,437457,58621,"[SAN BERNARDO, CALERA DE TANGO, BUIN, PAINE]",9,SAN BERNARDO,ORAL
33,REGION METROPOLITANA DE SANTIAGO,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,185966,93110,92856,114535,71431,"[MELIPILLA, SAN PEDRO, ALHUE, CURACAVI, MARIA ...",6,MELIPILLA,ORAL
34,REGION METROPOLITANA DE SANTIAGO,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL TALAGANTE,299830,148009,151821,254987,44843,"[TALAGANTE, EL MONTE, ISLA DE MAIPO, PEÑAFLOR,...",6,TALAGANTE,ORAL
40,REGION METROPOLITANA DE SANTIAGO,C.A. DE SAN MIGUEL,SEXTO TRIBUNAL DE JUICIO ORAL EN LO PENAL SANT...,1031854,502689,529165,1031854,0,"[LO ESPEJO, PEDRO AGUIRRE CERDA, SAN MIGUEL, S...",27,SANTIAGO,ORAL
113,REGION METROPOLITANA DE SANTIAGO,C.A. DE SAN MIGUEL,JUZGADO DE GARANTIA PUENTE ALTO,612816,298437,314379,590972,21844,"[PUENTE ALTO, SAN JOSE DE MAIPO, PIRQUE]",8,PUENTE ALTO,GARANTIA
114,REGION METROPOLITANA DE SANTIAGO,C.A. DE SAN MIGUEL,JUZGADO DE GARANTIA SAN BERNARDO,496078,244614,251464,437457,58621,"[SAN BERNARDO, CALERA DE TANGO, BUIN, PAINE]",10,SAN BERNARDO,GARANTIA
115,REGION METROPOLITANA DE SANTIAGO,C.A. DE SAN MIGUEL,JUZGADO DE GARANTIA MELIPILLA,139797,70058,69739,87508,52289,"[MELIPILLA, SAN PEDRO, ALHUE]",3,MELIPILLA,GARANTIA
116,REGION METROPOLITANA DE SANTIAGO,C.A. DE SAN MIGUEL,JUZGADO DE GARANTIA TALAGANTE,299830,148009,151821,254987,44843,"[TALAGANTE, EL MONTE, ISLA DE MAIPO, PEÑAFLOR,...",6,TALAGANTE,GARANTIA
117,REGION METROPOLITANA DE SANTIAGO,C.A. DE SAN MIGUEL,JUZGADO DE GARANTIA CURACAVI,46169,23052,23117,27027,19142,"[CURACAVI, MARIA PINTO]",2,CURACAVI,GARANTIA


## CARGA DATA A ARCHIVOS FUTHER

In [73]:
# Reset el index para realizar feather
df_tribunales_cortes.reset_index(inplace = True)
df_tribunales_poblacion.reset_index(inplace = True)
df_poblacion_jurisdiccion.reset_index(inplace=True)
# Directorio donde se guardaran archivos feather
os.makedirs(path_processed, exist_ok = True) 
df_tribunales_cortes.to_feather(f'{path_processed}/processes_ListadoTribunalesyCortes.feather')
df_tribunales_poblacion.to_feather(f'{path_processed}/processes_DataConsolidada_Poblacion_Tribunales.feather')
df_poblacion_jurisdiccion.to_feather(f'{path_processed}/processes_DataConsolidada_Poblacion_Jurisdiccion.feather')